## Target- urgent withdrawn by nurse(within 24hr)
* shift_applications status == 'cancelled' + shift_application prevStatus == 'confirmed'


### IDs -> only look at shifts application, leave each shifts after
* shift_applications id
* shift_applications user_id
* shift_applications shift_id


### Numeric Field
* shifts rate
* shifts net_pay: rate - nursedash profit
* shift_applications distance
* shift break time
* user withdrawn times:
* user applied times:
* user finished times: 


## Time
### shift, shift app time
* S_Create (shift created) => SA_Create (application created) => CWTime (comfim withdrawn) => Start_Time (shift start)
* S_Create: shift create time
* SA_Create: shift application create time
* CWTime: shift withdrawn time, whole confirmed withdrawn population (include within 24hr and > 24hr)
* Start_Time: shift start time
##### calculated field, graph in seconds, mins, hours
* S_Create2SA_Create: shift application create time - shift create time
* S_Create2Start_Time: shift start time - shift create time
* SA_Create2Start_Time: shift start time - shift application create time

### User
* user approvedAt:
* user createdAt:
#### Calculated field
* U_Create2U_approved: User approve time - User create time
* U_Create2now: now - User create time, in months(seconds/2629746)
* U_Approve2now: now - User approve time


### Categorical
* shifts role: position name + type
* facilities name = facilities short_name
* facilities areaId = facilities areaName
* withdrawnInfo_key, withdrawnInfo_value
* facilities segmentName: Senior Living = 1, Healthcare = 0
* Users enableNotifications:
* Users emailNotifications:
* Users appNotifications:
* Users allowedNotifications:


### ?
* facilities allowedQualification
* facilities createdAt
* facilities rates
* nurse shift withdrawn by admin, when policy start, how many withdrawn by admin
* shifts = unit
* role
* users updateAt ?
* users relationToFacility

# Data Cleaning

In [1]:
import pandas as pd

In [7]:
# df1 = pd.read_csv('0-2.csv')
# df2 = pd.read_csv('2-5.csv')
# df3 = pd.read_csv('5-9.csv')
# df4 = pd.read_csv('9-12.csv')
# df5 = pd.read_csv('12-18.csv')
# df = pd.concat([df1, df2, df3, df4, df5], axis = 0)
# df.to_csv('main.csv')

In [17]:
df = pd.read_csv('main.csv')

In [20]:
df.describe()

,Unnamed: 0,id,user_id,shift_id,distance,id.1,facility_id,assigned_nurse_id,rate,net_pay,...,SA_Create2Start_Time,S_Create2Start_Time,CW_Time2Start_Time,SA_Create2CW_Time,S_Create2CW_Time,areaId,U_create2now,U_approve2now,U_create2U_approve,tar
count,169654.000000,169654.000000,169654.000000,169654.000000,5.910900e+04,169654.000000,169654.000000,83226.000000,169654.000000,169654.000000,...,169654.000000,169654.000000,92453.000000,92453.000000,92453.000000,169654.000000,169654.000000,169618.000000,169618.000000,169654.000000
mean,18478.905861,97899.131261,8276.758915,62020.058325,8.195599e+04,62020.058325,270.774966,8004.185771,25.302433,19.522587,...,146.210969,234.022814,-86.617941,80.720274,176.412788,1.849623,13.110790,11.911528,36.068712,0.050803
std,12045.156778,39367.203740,4135.710825,31520.748998,1.975521e+05,31520.748998,165.091833,4048.920404,10.699482,9.172595,...,152.019559,211.797143,133.335933,121.110777,193.860430,1.140322,7.334501,6.596810,74.678177,0.219597
min,0.000000,25779.000000,18.000000,10837.000000,2.900000e+01,10837.000000,1.000000,18.000000,0.000000,0.000000,...,-8844.491944,-8844.488611,-1560.908889,0.000000,0.005556,1.000000,0.104826,0.076095,0.000104,0.000000
25%,8482.000000,64090.250000,4843.000000,32649.250000,2.016600e+04,32649.250000,132.000000,4674.000000,19.000000,13.750000,...,46.690625,90.704444,-118.048611,6.900556,45.183056,1.000000,6.433595,5.877229,0.611701,0.000000
50%,16965.000000,98335.000000,8557.000000,63916.000000,3.552400e+04,63916.000000,230.000000,8252.000000,21.500000,16.500000,...,103.734167,172.246944,-47.783889,39.336389,117.144722,1.000000,12.952611,11.467830,6.105694,0.000000
75%,27327.750000,131791.750000,11541.000000,87557.500000,5.858400e+04,87557.500000,407.000000,11165.000000,26.500000,21.000000,...,194.248333,314.083333,-14.323056,107.066389,241.429722,2.000000,19.518890,17.317081,29.439456,0.000000
max,49975.000000,168206.000000,17143.000000,120822.000000,2.947281e+06,120822.000000,608.000000,17143.000000,510.000000,84.000000,...,2112.624722,3064.164444,2879.511667,2969.024444,2971.023333,7.000000,31.272810,24.076044,673.477269,1.000000


In [13]:
df['tar'] = df.apply(lambda x: 1 if x['status'] == 'withdrawn' and x['prevStatus'] == 'confirmed' and x['withdrawnInfo_value'] == 'nurse' else 0, axis =1)

In [15]:
df['tar'].value_counts()

0    161035
1      8619
Name: tar, dtype: int64